In [31]:
# Import Splinter, BeautifulSoup, and Pandas

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
from time import sleep, ctime



In [32]:
# Path to chromedriver

path = "/usr/local/bin/chromedriver"


In [33]:
# Set the executable path and initialize the chrome browser in splinter

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [/Users/wolfey/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


## Visit the NASA mars news site

In [34]:
# Visit the mars nasa news site
nasa_url = "https://mars.nasa.gov/news/"

browser.visit(nasa_url)

# Optional delay for loading the page
sleep(5)


In [35]:
# Convert the browser html to a soup object and then quit the browser

html = browser.html
soup = bs(html, 'html.parser')



In [36]:
# .find() the content title and save it as `news_title`

article = soup.find_all('div', class_='content_title')
news_title = article[1].text
news_title


"NASA's Curiosity Rover Sends a Picture Postcard From Mars"

In [37]:
# .find() the paragraph text

paragraph_text = soup.find('div', class_='article_teaser_body').text
paragraph_text


'An artistic interpretation of Curiosity’s view high up on a Martian mountain was created by mission team members who were stunned by the sweeping landscape.'

## JPL Space Images Featured Image

In [38]:
# Visit JPL space images Mars URL 

jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)

In [39]:
# Find and click the full image button

browser.find_by_xpath('/html/body/div[1]/div/a/button').click()


In [40]:
# Find the more info button and click that


In [41]:
# Parse the resulting html with soup

html = browser.html
soup =  bs(html, 'html.parser')


In [42]:
# find the relative image url

relative_image_url = soup.find('img', class_="fancybox-image")["src"]
relative_image_url


'image/featured/mars1.jpg'

In [43]:
# Use the base url to create an absolute url
absolute_image_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{relative_image_url}'
absolute_image_url


'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'

## Mars Facts

In [53]:
# Create a dataframe from the space-facts.com mars page
mars = 'https://space-facts.com/mars/'
mars_df = pd.read_html(mars)
mars_df = mars_df[0]

mars_df


,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [54]:
# clean the dataframe and export to HTML

# Assign the columns
mars_df.columns = ['Description', 'Value']

# Set the index to the `Description`
mars_df.set_index('Description', inplace=True)

# Save html code
mars_df.to_html('table.html')

# Display mars_df
mars_df


,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


## Hemispheres

In [46]:
# visit the USGS astrogeology page for hemisphere data from Mars

usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(usgs_url)
html = browser.html
soup =  bs(html, 'html.parser')

In [47]:
# hemisphere = soup.find('h3')
# hemisphere.text
# hemisphere

In [48]:
# hemispheres = soup.find_all('h3')
# hemispheres


In [49]:
# list_of_hemispheres = []
# for hemisphere in hemispheres: 
#     list_of_hemispheres.append(hemisphere.text)
# list_of_hemispheres

In [50]:
# First, get a list of all of the hemispheres

# Parse the resulting html with soup
html = browser.html
soup =  bs(html, 'html.parser')

# List of hemispheres
h3 = soup.div.find_all('h3')
hemispheres = [i.text.replace(' Enhanced', '') for i in h3]
hemispheres

['Cerberus Hemisphere',
 'Schiaparelli Hemisphere',
 'Syrtis Major Hemisphere',
 'Valles Marineris Hemisphere']

In [51]:
# Next, loop through those links, click the link, find the sample anchor, return the href

list_of_hemispheres = []

for i in hemispheres:
    # Click into hemisphere page
    browser.links.find_by_partial_text(i).click()

    # We have to find the elements on each loop to avoid a stale element exception
    html = browser.html
    soup =  bs(html, 'html.parser')
    sample = soup.find('div', { "class" : "downloads"})
    # Next, we find the Sample image anchor tag and extract the href
    sample_image = sample.a['href']
    
    # Get Hemisphere title
    hemisphere_title = i

    # Append hemisphere object to list
    hemisphere_obj = {
        "img_url" : sample_image,
        "title"   : hemisphere_title
    }
    list_of_hemispheres.append(hemisphere_obj)
    
    # Navigate backwards with browser.back()
    browser.back()
    
# Quit the browser
browser.quit()

In [52]:
# view the hemisphere urls to make sure they look good
list_of_hemispheres

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere'}]